### Mangrove cover analysis
This notebooks calculates all mangrove cover statstics and outputs including change compared to the Global Mangrove Watch data from 1996.

In [1]:
import geopandas as gpd
import numpy as np
import os
import shutil

from rasterio.warp import Resampling

from config import Config
from core.analysis import calc_lucc_stats, clip_and_polygonize_raster, reproject_align_raster, reproject_raster
from core.utils import create_polygon, get_mapping_from_csv

In [2]:
config = Config.Config()

In [3]:
topo_mapping = get_mapping_from_csv(config.topo_legend_path, col_key="pixel", col_value="class_l2")
topo_cmap = get_mapping_from_csv(config.topo_legend_path, col_key="pixel", col_value="color_rgba", convert_rgba=True)
gmw_mapping = {0: "no mangrove", 1: "mangrove"}
pixel_area = config.gmw_resolution**2 / 1e6

## Merge, reproject and align rasters

In [4]:
if os.path.exists(config.analysis_mangrove_folder):
    shutil.rmtree(config.analysis_mangrove_folder)
os.makedirs(config.analysis_mangrove_folder)

In [5]:
# Reproject GMW raster to projected crs used for topo maps at its native resolution
reproject_raster(config.gmw_1996_merged_path, config.gmw_1996_proj_path, config.output_crs, resolution=config.gmw_resolution, resampling=Resampling.nearest)

In [6]:
# Reproject topo raster to same resolution and alignment as projected GLC raster
reproject_align_raster(config.raster_topo_l2_path, config.gmw_1996_proj_path, config.topo_aligned_gmw_path, resampling=Resampling.mode, colormap=topo_cmap)

Writing colormap


## Define study area and zones

In [7]:
# Load Vietnam EEz land union polygon
eez = gpd.read_file(config.vnm_eez_land_path)
vnm_eez = eez[eez["UNION"] == "Vietnam"].reset_index(drop=True).to_crs("EPSG:4326")

# Load map sheets
ms_index = gpd.read_file(config.map_sheet_index_geo_overedge_path).to_crs("EPSG:4326")
ms_area = ms_index.union_all()

# Load U Minh area polygon
u_minh = gpd.read_file(config.u_minh_path).to_crs("EPSG:4326").geometry[0]

# Create study area
study_area = vnm_eez.intersection(ms_area)
study_area_no_u_minh = study_area.difference(u_minh)

In [8]:
zone_ne = create_polygon(102, 20.674, 110, 22).intersection(study_area[0])
zone_nd = create_polygon(102, 19.883, 110, 20.674).intersection(study_area[0])
zone_sd = create_polygon(102, 7, 107.055, 12).intersection(study_area[0])
zone_cc = create_polygon(102, 7, 110, 19.883).intersection(study_area[0]).difference(zone_sd)
zone_sd_no_u_minh = zone_sd.difference(u_minh)

In [9]:
study_areas = gpd.GeoDataFrame({
    "region": ["North East", "Northern Delta", "Central Coast", "Southern Delta", "Southern Delta w/o U Minh", "Total", "Total w/o U Minh", "Vietnam Land + EEZ"],
    "geometry": [zone_ne, zone_nd, zone_cc, zone_sd, zone_sd_no_u_minh, study_area[0], study_area_no_u_minh[0], vnm_eez.geometry[0]]
}, crs="EPSG:4326").to_crs(config.output_crs)

In [10]:
study_areas.to_file(config.study_areas_mangrove_path, driver="GeoJSON")

## Aggregate mangrove changes by zones

In [11]:
res_mangrove_change = calc_lucc_stats(
    study_areas,
    index_cols="region",
    src_raster=config.topo_aligned_gmw_path,
    dst_raster=config.gmw_1996_proj_path,
    src_mapping=topo_mapping,
    dst_mapping=gmw_mapping,
    src_class="mangrove",
    dst_class="mangrove",
    pixel_area=pixel_area
)
res_mangrove_change.drop("Vietnam Land + EEZ").drop("geometry", axis=1).to_csv(config.mangrove_stats_study_area, index=False)
np.round(res_mangrove_change.drop("Vietnam Land + EEZ"), 1)

,geometry,region,area,src_perc,dst_perc,change_perc_total_area,change_perc_src_area,src_pixels,dst_pixels,src_area,dst_area,change_area
region,,,,,,,,,,,,
North East,"POLYGON ((-2125010.411 4039142.914, -2125079.3...",North East,69289.5,0.5,0.3,-0.2,-40.5,108478251.3,108478251.3,359.2,213.9,-145.3
Northern Delta,"MULTIPOLYGON (((-2456001.972 3884000.515, -245...",Northern Delta,18372.1,0.0,0.4,0.4,3532.3,29395282.6,29395282.6,2.0,71.2,69.2
Central Coast,"POLYGON ((-2160437.116 3812299.295, -2189199.1...",Central Coast,174871.8,0.1,0.0,-0.0,-66.2,279794904.4,279794904.4,122.3,41.3,-81.0
Southern Delta,"MULTIPOLYGON (((-2016294.82 3014023.182, -2004...",Southern Delta,65730.5,5.9,2.4,-3.4,-58.4,105168760.2,105168760.2,3852.6,1601.2,-2251.4
Southern Delta w/o U Minh,"MULTIPOLYGON (((-2004252.254 2826000.546, -201...",Southern Delta w/o U Minh,63907.5,4.0,2.5,-1.5,-37.7,102252039.6,102252039.6,2568.5,1601.0,-967.4
Total,"MULTIPOLYGON (((-2364610.605 2817148.898, -236...",Total,368938.7,1.3,0.6,-0.7,-55.5,548907211.8,548907211.8,4336.0,1927.5,-2408.5
Total w/o U Minh,"MULTIPOLYGON (((-2363261.778 2817726.593, -236...",Total w/o U Minh,367115.7,0.9,0.6,-0.3,-36.8,545990491.2,545990491.2,3051.9,1927.4,-1124.6


In [12]:
# Area and percentage of mangrove forests (based on GMW data 1996) not covered by the study area
coverage_diff = res_mangrove_change.loc["Vietnam Land + EEZ", "dst_area"] - res_mangrove_change.loc["Total", "dst_area"]
coverage_diff_perc = coverage_diff / res_mangrove_change.loc["Vietnam Land + EEZ", "dst_area"]
print("Area not covered", np.round(coverage_diff, 1), "km2")
print("Percent not covered", np.round(coverage_diff_perc * 100, 1), "%")

Area not covered 34.3 km2
Percent not covered 1.7 %


# Aggregate mangrove changes by map sheet

In [13]:
# Load map sheet index
ms_index = gpd.read_file(config.map_sheet_index_geo_overedge_path).to_crs(config.output_crs)
ms_index = ms_index[["key", "edition", "map_info_date", "legend_type", "geometry"]]

# Crop to mangrove study area 
ms_index.geometry = ms_index.geometry.intersection(study_area_no_u_minh.to_crs(config.output_crs)[0])
ms_index = ms_index[ms_index.geometry.is_empty == False]

In [14]:
# Calculate forest cover change stats per map sheet
ms_index = calc_lucc_stats(
    ms_index,
    index_cols="key",
    src_raster=config.topo_aligned_gmw_path,
    dst_raster=config.gmw_1996_proj_path,
    src_mapping=topo_mapping,
    dst_mapping=gmw_mapping,
    src_class="mangrove",
    dst_class="mangrove",
    pixel_area=pixel_area
)

ms_index["mangrove_loss"] = ms_index["change_area"].apply(lambda x: -x if x < 0 else 0)
ms_index["mangrove_gain"] = ms_index["change_area"].apply(lambda x: x if x > 0 else 0)

In [15]:
ms_index.to_file(config.mangrove_stats_map_sheets, driver="GeoJSON", index=False)

/home/s1835403/datastore/.conda/envs/topo-maps/lib/python3.12/site-packages/pyogrio/raw.py:723: RuntimeWarning: NaN of Infinity value found. Skipped
  ogr_write(


## Create vector files for figures showing stable, lost and gained areas

In [16]:
# Mangrove area
mangrove_topo = clip_and_polygonize_raster(config.topo_aligned_gmw_path, config.mangrove_raster_idx, study_area[0], "EPSG:4326")
print(mangrove_topo.area.sum()/1e6)
mangrove_topo.to_crs("EPSG:4326").to_file(config.mangrove_topo_vector)

4336.02


In [17]:
mangrove_gmw_1996 = clip_and_polygonize_raster(config.gmw_1996_proj_path, 1, study_area[0], "EPSG:4326")
print(mangrove_gmw_1996.area.sum()/1e6)
mangrove_gmw_1996.to_crs("EPSG:4326").to_file(config.mangrove_gmw_vector, driver="GeoJSON")

1927.530625


In [18]:
# Compute intersection (original area - present in both years)
area_stable = gpd.overlay(mangrove_gmw_1996, mangrove_topo, how="intersection")

# Compute area gained (present now but not in 1996)
area_gained = gpd.overlay(mangrove_gmw_1996, mangrove_topo, how="difference")

# Compute area lost (present in 1996 but not now)
area_lost = gpd.overlay(mangrove_topo, mangrove_gmw_1996, how="difference")

# save outputs
area_stable.to_crs("EPSG:4326").to_file(config.mangrove_stable_path)
area_lost.to_crs("EPSG:4326").to_file(config.mangrove_losses_path)
area_gained.to_crs("EPSG:4326").to_file(config.mangrove_gains_path)

/tmp/ipykernel_2455386/2226166242.py:2: UserWarning: `keep_geom_type=True` in overlay resulted in 7877 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  area_stable = gpd.overlay(mangrove_gmw_1996, mangrove_topo, how="intersection")
